## Wstęp

W tym notebooku przedstawimy czym jest NeRF, do czego służy oraz jakie techniki są używane aby usprawnić jego działanie.

#### Czym jest NeRF ?

NeRF to akronim od Neural Radiance Fields for View Synthesis, co można przetłumaczyć jako Neuronowe Pola Radiacyjne do syntezy widoków.

Mówiąc bardziej zrozumiale, możemy powiedzieć że NeRF to sieć fully-connected która bedzie reprezentować obraz 3D w taki sposób że po podaniu jako input współrzędnych obserwatora oraz kąty patrzenia dostaniemy jako output zdjęcie z perspektywy obserwatora.



<div style="padding: 40px;">
<img src="src/f1.png" width="2000" />
</div>


## W jaki sposób NeRF przetwarza input

Reprezentujemy ciągłą scenę jako funkcję wektorową o wartościach 5D, której dane wejściowe to współrzędne (x,y,z) i 2D kierunek widzenia (θ, φ), a wyjście to emitowany kolor 

Przybliżamy tę ciągłą reprezentację sceny 5D za pomocą sieci :
<br>
### $ MLP - FΘ: (x, d) → (c, σ) $

#### $x = $ (x, y, z) 

#### $d -$ kierunek    wektora     w   przestrzeni  trójwymiarowej

#### $c =$ (r, g, b) 

#### $σ -$ gęstośc punktów / gęstośc objętości ( jak dużo w danym punkcie znajduje sie materii )


i optymalizujemy jej wagi Θ, aby mapować z każdej wejściowej współrzędnej 5D do odpowiedniej gęstości objętości i emitowanego kierunkowego koloru. Aby to osiągnąć, MLP FΘ najpierw przetwarza dane wejściowe 3D koordynatu x za pomocą 8 w pełni połączonych warstw (używając aktywacji ReLU i 256 kanałów na warstwę), a następnie wyjścia σ i 256-wymiarowego wektora cech. Ten wektor cech następnie jest łączony z kierunkiem widzenia promienia kamery i przekazywany do dodatkowej, w pełni połączonej warstwy (używając aktywacji ReLU i 128 kanałów), która wyjściowo emituje zależny od widzenia kolor RGB.

<div style="padding: 40px;">
<img src="src/nerf_layers.png" width="2000" />
</div>


## Pytanie 1
### Dlaczego dodawany jest wektor w przestrzeni 3D ( oznaczenie jako d ) jako input do tej sieci?

<---- Miejsce na odpowiedź ---->

## W jaki sposób NeRF renderuje obrazy?
NeRF tworzy obraz 2D za pomocą techniki zwanej renderowaniem volumetrycznym. Ta technika tworzy obraz przez śledzenie promienia (linii) przechodzącej przez każdy piksel wirtualnej kamery, przechodzącego przez scenę. Dane jest to następującym wzorem:

( można o tym wyżej wzorze myśleć jak o wartości oczekiwanej koloru biorąc pod uwage zakres długości promienia, koloru powierzchni przez które przechodzi promień oraz gestości obiektu przez który promień przechodzi )



<div style="padding: 20px;">
<img src="src/render.png" width="1000" />
</div>
<br>
<div style="padding: 10px;">
<img src="src/dlugosc_toru_promienia.png" width="500" />
</div>


tn - dolny zakres długości promienia
tf - górny zakres długości promienia
r(t) - koniec wektora zaczepionego w punkcie o ( czyli pozycja obserwatora ) o zasiegu wektora t oraz kierunku wektora danym przez d
<br>
<div style="padding: 10px;">
<img src="src/promienie_gestosc.png" width="1000" />
</div>

Rysunek poglądowy może pomóc zrozumieć jak gęstośc otoczenia i odległość promienia może wpływać na kolor oraz jak ten kolor potem używać do obliczania loss'u (g.t. - ground truth ;) )

## Pytanie 2
### Opisz swoimi słowami co robią poszczególne czynniki w poniższym wzorze na kolor?

<div style="padding: 20px;">
<img src="src/render.png" width="1000" />
</div>
<br>
<div style="padding: 10px;">
<img src="src/dlugosc_toru_promienia.png" width="500" />
</div>

<---- Miejsce na odpowiedź ---->



## Usprawnienie modelu

#### Jedną z najwazniszych rzeczy jakie można zrobić to Fourier Feature

Operowanie na suchych współrzednych x,y,z przynosi słabe rezultaty w kontekście szczegółów i kolorów na obrazku, autorzy pracy powołując sie na innych badaczy dokonują następującej transformacji:

<div style="padding: 10px;">
<img src="src/fourier_feature.png" width="2000" />
</div>

gdzie 
$p = x * B$
<br>
$B -$ macierz gausowska z rozkładu `N(0, σ^2)`

Poniższy przykład pokazuje jak może wygląda nauka z użyciem Fourier Feature i bez niego, gołym okiem widać na obrazku oraz na lossie różnice

<div style="padding: 10px;">
<img src="src/lion_none_gauss_v1.gif" width="2000" />
</div>

Należy jednak uważać, ponieważ nieodpowiednie dobranie parametrów może doprowadzić do pogorszenia jakości zdjeć, tyczy sie to parametru L

<div style="padding: 10px;">
<img src="src/test_sweep_1e-4_5000_more_low.gif" width="2000" />
</div>

## Proces trenowania

Trenowanie tej sieci można przedstawić w następujących krokach:

1. Odpowienie przygotowanie danych. Bedziemy potrzebować wykonanych zdjęć wraz z pozcją kamery.
2. W każdej iteracji jest losowany zestaw promieni dla których bedzie liczony kolor
3. Dla każdego wylosowanego promienia jest obliczanych N wylosowanych próbek na całej jego długości
4. Z wylosowanych próbek za pomocą renderowania objętości jest obliczany kolor
5. Ostatnim krokiem jest obliczanie loss'u który jest dany wzorem: 

<div style="padding: 10px;">
<img src="src/loss.png" width="500" />
</div>


## Rezultaty

<div style="padding: 10px;">
<img src="src/res3.png" width="2000" />
</div>

<div style="padding: 10px;">
<img src="src/res2.png" width="2000" />
</div>

<div style="padding: 10px;">
<img src="src/res1.png" width="2000" />
</div>


Zródła:

https://www.matthewtancik.com/nerf
https://dl.acm.org/doi/pdf/10.1145/3503250
https://towardsdatascience.com/nerf-representing-scenes-as-neural-radiance-fields-for-view-synthesis-ef1e8cebace4
https://bmild.github.io/fourfeat/
https://arxiv.org/pdf/2006.10739.pdf